In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import time
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
from tqdm import tqdm
import os
import scipy.io
import scipy.misc
from skimage.transform import resize

# DOWNLOAD THE FLOW AROUND THE CYLINDER DATA
plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams.update({'font.size': 14})
                    
                    
# Load Cylinder flow data set
mat_contents = scipy.io.loadmat(os.path.join('CYLINDER_ALL'))
vortall = mat_contents['VORTALL']

m, n = *mat_contents['m'], *mat_contents['n']

print("number of datapoints:", m)
print("number of snapshots:", n)

In [ ]:
#USING THE FIRST P% OF THE TEMPORAL SNAPSHOT, FORECAST THE REMAING (100-P)% FUTURE STATE DATA. 
p = 0.5
train = vortall[:, :int(vortall.shape[1]*p)] # FIRST P% OF THE TEMPORAL SNAPSHOT
test = vortall[:, int(vortall.shape[1]*p)-1:] # (100-P)% FUTURE STATE DAT

In [ ]:
# DO THIS BY TRAINING A NEURAL NETWORK ON THE HIGH- DIMENSIONAL DATA AND USING
# 1) A FEEDFOWARD NN

class FeedForward(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
                                nn.Linear(self.input_dim, self.input_dim),
                                nn.ReLU(),
                                nn.Linear(self.input_dim, self.input_dim)
        )

    def forward(self, x):
        return self.net(x)

    
class Model(FeedForward):
    def __init__(self, shapes, learn_rate=1e-2):
        self.input_dim = shapes[0]
        self.num_chunks = shapes[1]
        self.learn_rate = learn_rate
        super().__init__()
        self.optimizer = torch.optim.Adam(self.net.parameters(), lr=self.learn_rate)
        self.chunk_pred = []
        self.loss = []
        self.validation = []

    
    
    def chunks(self, max_layer_input, divider):
        return [(i*max_layer_input//divider, (i+1)*max_layer_input//divider) for i in range(divider)]
    
    def train(self, data_loader, tot_iters):
        for tot_iter in tqdm(range(tot_iters)):
            for tot_samp_idx, (train, test) in enumerate(data_loader):
                for (start, end) in self.chunks(self.input_dim*self.num_chunks, self.num_chunks):
                    train_chunk, test_chunk = train[start:end].float(), test[start:end].float()
                    net_real = self.net(train_chunk)
                    self.chunk_pred.append(net_real)

                    self.optimizer.zero_grad()
                    loss = nn.MSELoss()(net_real, test_chunk)
                    loss.backward()
                    self.optimizer.step()

                    
                self.loss.append(nn.MSELoss()(torch.ravel(torch.stack(self.chunk_pred)), test).data.item())
                self.chunk_pred = []
                    
    def predict(self, data_loader):
        self.chunk_pred = []
        for pair_idx, (X_test, Y_test) in enumerate(data_loader):
            for (start, end) in self.chunks(self.input_dim*self.num_chunks, self.num_chunks):
                X_test_chunk, Y_test_chunk = X_test[start:end].float(), Y_test[start:end].float()
                net_real = self.net(X_test_chunk)
                self.chunk_pred.append(net_real)

            self.validation.append(nn.MSELoss()(torch.ravel(torch.stack(self.chunk_pred)), Y_test).data.item())
            self.chunk_pred = []
    
                
    def plot(self):
        f, ax = plt.subplots(1, 2, figsize=(10, 5))
        ax[0].plot(self.loss, c='b')
        ax[0].set_title('Training Data MSE')
        ax[0].set_xlabel('Epoch * Tot_Sample')
        ax[1].plot(self.validation, c='r')
        ax[1].set_title('Testing Data MSE')
        ax[1].set_xlabel('Pair index');

In [ ]:
torch.manual_seed(0)
train_loader = torch.utils.data.DataLoader(train.T, batch_size=2)
torch.manual_seed(0)
test_loader = torch.utils.data.DataLoader(test.T, batch_size=2, shuffle=False)

torch.manual_seed(0)
net = Model(shapes=(int(*n),int(*m)), learn_rate=1e-3)
net.train(train_loader, 40)
net.train(test_loader)
net.plot()


In [ ]:
# REDO THE FORECASTING CALCULATIONS BY TRAINING A MODEL IN THE REDUCED SUBSPACE U FROM THE SVD
#EVALUATE THE FORECASTING PERFORMANCE AS A FUNCTION OF THE PERCENTAGE OF THE TRAINING DATA P AND THE RANK OF THE REDUCED SPACE rU, S, VT = np.linalg.svd(vortall, full_matrices=False)

p=0.5
train = U[:, :int(U.shape[1]*p)]
test = U[:, int(U.shape[1]*p)-1:]

torch.manual_seed(0)
train_loader_2 = torch.utils.data.DataLoader(train.T, batch_size=2, shuffle=False)


torch.manual_seed(0)
net = Model(shapes=(int(*n),int(*m)), learn_rate=1e-3)
net.train(train_loader_2, 40)



torch.manual_seed(0)
test_loader_2 = torch.utils.data.DataLoader(test.T, batch_size=2, shuffle=False)
net.predict(test_loader_2)


net.plot()
